Phase 1 – OCR + Text-based Retrieval (RAG Skeleton)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.9/425.9 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
pip install tools

In [ ]:
!pip install PyMuPDF pytesseract Pillow numpy sentence-transformers pinecone-client gradio transformers torch torchvision torchaudio accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import numpy as np
import io
from sentence_transformers import SentenceTransformer
import pinecone
import gradio as gr
from transformers import DonutProcessor, VisionEncoderDecoderModel, pipeline
import torch
import os

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_4uyLm9_67ctzCZMhy4tZExs4Kkf5Bd8DjherbtDMNz5gWJ3JkZytkkV3F1PfQkVRD5uvFh"

Required Configuration

In [ ]:
# CONFIG
OCR_LANG = 'eng'
EMBED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

In [ ]:
# Load embedding model
embedder = SentenceTransformer(EMBED_MODEL_NAME)
vector_dim = embedder.get_sentence_embedding_dimension()

In [ ]:
llm_pipe = pipeline("text2text-generation", model="google/flan-t5-base")

Device set to use cuda:0


In [ ]:
pinecone_api_key = "pcsk_4uyLm9_67ctzCZMhy4tZExs4Kkf5Bd8DjherbtDMNz5gWJ3JkZytkkV3F1PfQkVRD5uvFh"
vector_dim = 384


In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Define your key and dimensions
pinecone_api_key = "pcsk_4uyLm9_67ctzCZMhy4tZExs4Kkf5Bd8DjherbtDMNz5gWJ3JkZytkkV3F1PfQkVRD5uvFh"
vector_dim = 384  # or 768 depending on your embedding model

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Define index name
index_name = "pdf-rag-index"

# Check and create index with ServerlessSpec
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=vector_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",         # or "gcp"
            region="us-east-1"   # or "gcp-starter", etc., based on your Pinecone environment
        )
    )

# Connect to index
index = pc.Index(index_name)


In [ ]:
# STORAGE
text_chunks = []
meta_info = []
current_pages = []

def extract_text_and_images(pdf_bytes):
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    extracted = []
    for page_num, page in enumerate(doc):
        text = page.get_text()
        if text.strip():
            extracted.append((f"Page {page_num+1}", text))
        # OCR for images
        pix = page.get_pixmap()
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        ocr_text = pytesseract.image_to_string(img, lang=OCR_LANG)
        if ocr_text.strip():
            extracted.append((f"Page {page_num+1} (OCR)", ocr_text))
    return extracted, [page.get_pixmap().tobytes("png") for page in doc]

In [ ]:
def chunk_text(text, size=300):
    words = text.split()
    chunks = [" ".join(words[i:i+size]) for i in range(0, len(words), size)]
    return chunks

In [ ]:
def build_pinecone_index(extracted):
    global text_chunks, meta_info
    text_chunks.clear()
    meta_info.clear()
    to_upsert = []
    for i, (page_id, txt) in enumerate(extracted):
        for ch in chunk_text(txt):
            chunk_id = f"chunk-{len(text_chunks)}"
            text_chunks.append(ch)
            meta_info.append(page_id)
            emb = embedder.encode(ch).tolist()
            to_upsert.append((chunk_id, emb, {"page": page_id}))
    if to_upsert:
        index.upsert(vectors=to_upsert)

In [ ]:
def retrieve_answer(query, top_k=3):
    if len(text_chunks) == 0:
        return "No data indexed yet. Upload PDFs first."
    q_emb = embedder.encode(query).tolist()
    result = index.query(vector=q_emb, top_k=top_k, include_metadata=True)
    results = []
    for match in result["matches"]:
        meta = match["metadata"].get("page", "")
        chunk_id = int(match["id"].split("-")[1])
        results.append(f"[{meta}]: {text_chunks[chunk_id]}")
    return "\n---\n".join(results)

def vision_qa(image_bytes, question):
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    task_prompt = f"<s_docvqa><s_question>{question}</s_question><s_answer>"
    decoder_input_ids = donut_processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
    pixel_values = donut_processor(image, return_tensors="pt").pixel_values
    outputs = donut_model.generate(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids, max_length=512)
    answer = donut_processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return answer

def synthesize_answer(query, top_k=3):
    retrieved = retrieve_answer(query, top_k)
    prompt = f"Context: {retrieved}\n\nQuestion: {query}\nAnswer concisely:"
    ans = llm_pipe(prompt, max_length=512)[0]['generated_text']
    return ans

In [ ]:
def upload_pdfs(files):
    global current_pages
    all_extracted = []
    current_pages = []

    for f in files:
        with open(f, "rb") as file:
            bytes_data = file.read()
        extracted, images = extract_text_and_images(bytes_data)
        all_extracted.extend(extracted)
        current_pages.extend(images)

    build_pinecone_index(all_extracted)
    return f"Processed {len(all_extracted)} chunks from {len(files)} PDFs."


In [ ]:
# GRADIO UI

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📄 Multi-PDF ChatApp AI Agent (with Pinecone)")
    with gr.Row():
        file_input = gr.File(file_types=['.pdf'], label="Upload PDFs", file_count="multiple")
        status = gr.Textbox(label="Processing Status")
    process_btn = gr.Button("Process PDFs")
    with gr.Tab("Ask a Question"):
        with gr.Row():
            question = gr.Textbox(label="Ask a Question")
            use_vision = gr.Checkbox(label="Use Vision Transformer (Donut)")
            page_num = gr.Number(value=0, precision=0, label="Page Number (for Vision QA)")
        answer = gr.Textbox(label="Answer")
    process_btn.click(upload_pdfs, inputs=file_input, outputs=status)
    question.submit(ask_question_ui, inputs=[question, use_vision, page_num], outputs=answer)

In [ ]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5ce79385c8f98a8bb2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5ce79385c8f98a8bb2.gradio.live
